In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.nonparametric.smoothers_lowess import lowess
import numpy as np
import scipy.stats as stats
from statsmodels.formula.api import ols

In [ ]:
#import datset
CGSS2017 = pd.read_csv (r'cgss2017.csv', low_memory=False)

In [ ]:
print(f"Data frame is {CGSS2017.shape[0]:,} x {CGSS2017.shape[1]}")

In [ ]:
#define needed variables
cols=['a31','a7a','a15','a8a','a43a','a89b','a27h','a2','a43d']

In [ ]:
#subset with needed variables
CGSS2017_1 = pd.read_csv (r'cgss2017.csv', low_memory=False, usecols=cols)
print(f"Data frame is {CGSS2017_1.shape[0]:,} x {CGSS2017_1.shape[1]}")

In [ ]:
CGSS2017_1.sample(6, random_state=8)

In [ ]:
#rename columnes
CGSS2017_1 = CGSS2017_1.rename(columns = {'a2':'gender','a31':'birth_year','a7a':'c_educ','a8a':'c_income',
                         'a15':'c_health','a27h':'c_hukou','a43a':'c_sc',
                                         'a43d':'p_sc','a89b':'f_educ'})

In [ ]:
#subset and check for numbers people born from 1960-1979, total value, boys and girls
n1 = CGSS2017_2[(CGSS2017_2['birth_year'] > 1959) & (CGSS2017_2['birth_year'] < 1980) ]
n1n = n1.shape[0]
n1b = n1[n1['gender'].isin(['1'])].shape[0]
n1g = n1[n1['gender'].isin(['2'])].shape[0]
print(n1n, n1b, n1g)

In [ ]:
#subset and check for numbers people born from 1980-1999, boys and girls
n2 = CGSS2017_2[CGSS2017_2['birth_year'] > 1979]
n2n = n2.shape[0]
n2b = n2[n2['gender'].isin(['1'])].shape[0]
n2g = n2[n2['gender'].isin(['2'])].shape[0]
print(n2n, n2b, n2g)

In [ ]:
#missing value in n1
c_sc_1 = n1[n1['c_sc'].isin(['98','99'])].shape[0]
p_sc_1 = n1[n1['p_sc'].isin(['98','99'])].shape[0]
c_health_1 = n1[n1['c_health'].isin(['98','99'])].shape[0]
c_educ_1 = n1[n1['c_educ'].isin(['14'])].shape[0]
f_educ_1 = n1[n1['f_educ'].isin(['14','98','99'])].shape[0]
c_income_1 = n1[n1['c_income'].isin(['9999996','99999997','99999998','99999999'])].shape[0]
c_hukou_1 = n1[n1['c_hukou'].isin(['98','99'])].shape[0]
birth_year_1 = n1[n1['birth_year'].isin(['98'])].shape[0]
print(c_sc_1, p_sc_1,c_health_1, c_educ_1, f_educ_1, c_income_1, c_hukou_1, birth_year_1)

In [ ]:
#missing value in n2
c_sc_2 = n2[n2['c_sc'].isin(['98','99'])].shape[0]
p_sc_2 = n2[n2['p_sc'].isin(['98','99'])].shape[0]
c_health_2 = n2[n2['c_health'].isin(['98','99'])].shape[0]
c_educ_2 = n2[n2['c_educ'].isin(['14'])].shape[0]
f_educ_2 = n2[n2['f_educ'].isin(['14','98','99'])].shape[0]
c_income_2 = n2[n2['c_income'].isin(['9999996','99999997','99999998','99999999'])].shape[0]
c_hukou_2 = n2[n2['c_hukou'].isin(['98','99'])].shape[0]
birth_year_2 = n2[n2['birth_year'].isin(['98'])].shape[0]
print(c_sc_2, p_sc_2,c_health_2, c_educ_2, f_educ_2, c_income_2, c_hukou_2, birth_year_2)

In [ ]:
#export to csv for R to do hot deck imputation
n1.to_csv(r'n1',index=False)
n2.to_csv(r'n2',index=False)

In [ ]:
#import dataset after HDI
n1_imp = pd.read_csv (r'n1_imp.csv')
n2_imp = pd.read_csv (r'n2_imp.csv')
n2_imp.sample(6, random_state=8)

In [ ]:
#transform hukou to dummy variables
# creating dummies for gender 
n1_imp['c_hukou'] = n1_imp['c_hukou'].map({2:1, 3:1, 4:2, 5:2, 6:2,7:2}) 
n2_imp['c_hukou'] = n2_imp['c_hukou'].map({2:1, 3:1, 4:2, 5:2, 6:2,7:2}) 
n1_imp.sample(6, random_state=8)

In [ ]:
#find mean of each gender
n1_imp.groupby('gender').mean()

In [ ]:
#find mean of each gender
n2_imp.groupby('gender').mean()

In [ ]:
#find std n1
n1_imp.groupby('gender').std()

In [ ]:
#find std n2
n2_imp.groupby('gender').std()

In [ ]:
# find median of children income
m1 = n1_imp.groupby('gender').median()
m2 = n2_imp.groupby('gender').median()
print(m1, m2)

In [ ]:
#residual plot
#fit simple linear regression model
model = ols('c_educ ~ f_educ+ p_sc + c_sc + c_hukou +c_health', data=n1_imp).fit()

#view model summary
print(model.summary())

In [ ]:
#partial regression plot
fig = sm.graphics.plot_partregress_grid(model)
fig.tight_layout(pad=0.35)

In [ ]:
#residual plot
view_rawresiduals = model.resid
fitted = model.fittedvalues
smoothed = lowess(view_rawresiduals,fitted)
top3 = abs(view_rawresiduals).sort_values(ascending = False)[:3]

plt.rcParams.update({'font.size': 16})
plt.rcParams["figure.figsize"] = (8,7)
fig, ax = plt.subplots()
ax.scatter(fitted, residuals, edgecolors = 'k', facecolors = 'none')
ax.plot(smoothed[:,0],smoothed[:,1],color = 'r')
ax.set_ylabel('Residuals')
ax.set_xlabel('Fitted Values')
ax.set_title('Residuals vs. Fitted')
ax.plot([min(fitted),max(fitted)],[0,0],color = 'k',linestyle = ':', alpha = .3)

for i in top3.index:
    ax.annotate(i,xy=(fitted[i],residuals[i]))

plt.show()

In [ ]:
#QQplot
sorted_student_residuals = pd.Series(model.get_influence().resid_studentized_internal)
sorted_student_residuals.index = model.resid.index
sorted_student_residuals = sorted_student_residuals.sort_values(ascending = True)
df = pd.DataFrame(sorted_student_residuals)
df.columns = ['sorted_student_residuals']
df['theoretical_quantiles'] = stats.probplot(df['sorted_student_residuals'], dist = 'norm', fit = False)[0]
rankings = abs(df['sorted_student_residuals']).sort_values(ascending = False)
top3 = rankings[:3]

fig, ax = plt.subplots()
x = df['theoretical_quantiles']
y = df['sorted_student_residuals']
ax.scatter(x,y, edgecolor = 'k',facecolor = 'none')
ax.set_title('Normal Q-Q')
ax.set_ylabel('Standardized Residuals')
ax.set_xlabel('Theoretical Quantiles')
ax.plot([np.min([x,y]),np.max([x,y])],[np.min([x,y]),np.max([x,y])], color = 'r', ls = '--')
for val in top3.index:
    ax.annotate(val,xy=(df['theoretical_quantiles'].loc[val],df['sorted_student_residuals'].loc[val]))
plt.show()

In [ ]:
#fit OLS regression model for first cohort,education
c_educ_1b = n1_imp.loc[n1_imp['gender'] == 1]
c_educ_1b = ols('c_educ ~ f_educ+ p_sc + c_hukou +c_health + birth_year', data=c_educ_1b).fit()
c_educ_1g = n1_imp.loc[n1_imp['gender'] == 2]
c_educ_1g = ols('c_educ ~ f_educ+ p_sc + c_hukou +c_health+ birth_year', data=c_educ_1g).fit()
#view model summary
print(c_educ_1b.summary())
print(c_educ_1g.summary())

In [ ]:
#fit OLS regression model for first cohort,education
c_educ_2b = n2_imp.loc[n2_imp['gender'] == 1]
c_educ_2b = ols('c_educ ~ f_educ+ p_sc + c_hukou +c_health + birth_year', data=c_educ_2b).fit()
c_educ_2g = n2_imp.loc[n2_imp['gender'] == 2]
c_educ_2g = ols('c_educ ~ f_educ+ p_sc + c_hukou +c_health+ birth_year', data=c_educ_2g).fit()
#view model summary
print(c_educ_2b.summary())
print(c_educ_2g.summary())

In [ ]:
#fit OLS regression model for first cohort,social class
c_sc_1b = n1_imp.loc[n1_imp['gender'] == 1]
c_sc_1b = ols('c_sc ~ f_educ+ p_sc + c_hukou +c_health + c_educ+ birth_year', data=c_sc_1b).fit()
c_sc_1g = n1_imp.loc[n1_imp['gender'] == 2]
c_sc_1g = ols('c_sc ~ f_educ+ p_sc + c_hukou +c_health + c_educ+ birth_year', data=c_sc_1g).fit()
#view model summary
print(c_sc_1b.summary())
print(c_sc_1g.summary())

In [ ]:
#fit OLS regression model for first cohort,social class
c_sc_2b = n2_imp.loc[n2_imp['gender'] == 1]
c_sc_2b = ols('c_sc ~ f_educ+ p_sc + c_hukou +c_health + c_educ+ birth_year', data=c_sc_2b).fit()
c_sc_2g = n2_imp.loc[n2_imp['gender'] == 2]
c_sc_2g = ols('c_sc ~ f_educ+ p_sc + c_hukou +c_health + c_educ+ birth_year', data=c_sc_2g).fit()
#view model summary
print(c_sc_2b.summary())
print(c_sc_2g.summary())